In [15]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t, get_h1_spin, get_g_spin
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, remain_only_tridiagonal_elements
from pytenet.global_krylov_method import generate_Hamiltonian_with_occupation_number, generate_reduced_H_non_ortho, coeff_gram_schmidt, solve_ritz
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn


Load and initialize datas: 

no is number of spatial orbitals

L is number of spatial orbitals, L = no

We DON'T need t_spin, g_spin now, only 'original' datas are enough 

r_THC is THC rank

In [16]:
#load integrals
with h5py.File("../data_water/eri_water.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("../data_water/x.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

7
(7, 28)
(28, 28)
rl errV: 2.8386751875274264e-12
abs errV: 2.0615721155266396e-11
errt: 7.097049412242525e-13
errh: 2.585427402664151e-13
errht: 9.079449636842276e-14


These Hamiltonian are exact molecular Hamiltonian and molecular Hamiltonian reconstructed by THC tensors. The calculation cost time, so that we store them in disk and load them when needed. For water molecule H2O in STO-6G basis, the error is small for r_THC = 28.

Actually, considering there are always 10 electrons for a water molecule, we only retain the elements which operator quantum states with 10 electrons.

In [17]:
#load Hamiltonian generated by above coefficients
# H_correct = scipy.io.mmread('data_water/H_water_correct.mtx').tocsr()
# H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)
# e, v = sparse.linalg.eigsh(H_correct_10e, which = 'SA', k = 10)
# e_ground = e[0]
# e_1st_ex = e[1]

e_ground = -84.92262983120877
e_1st_ex = -84.52780562388604
e_2nd_ex = -84.46820215626565
e_3rd_ex = -84.42486181064207
e_4th_ex = -84.42379006099287

To construct 'original mpo' of molecular Hamiltonian, currently we can use ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy), which consider the spins automatically, we don't need generating spin-version datas anymore. The result of ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy) is mpo whose physical dimension is 4.

In [18]:
# h1_spin = get_h1_spin(h1)
# g_spin = get_g_spin(eri)
g_phy =  eri.transpose(0, 2, 1, 3)
#mpo_ref = ptn.hamiltonian.molecular_hamiltonian_mpo(h1_spin, g_spin_phy)
mpo_ref = ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy)
print(mpo_ref.bond_dims)

[1, 16, 58, 96, 96, 58, 16, 1]


Generate Hartree-Fock state. See function generate_single_state for further details.

In [19]:
HFS = generate_single_state(7, [3, 3, 3, 3, 3, 0, 0])

Generate THC-MPO by layers, using THC tensors. 
It returns a list of H_mu_nu, each H_mu_nu is also a list, which contains four smaller MPOs with bond dims 2.
The final element of this list is MPO for kinetic term.
The physical bond_dim is 4.

In [20]:
#generate thc_mpo
t = get_t(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t)

print(type(H_mu_nu_list_spin_layer))
print(type(H_mu_nu_list_spin_layer[0]))
print(type(H_mu_nu_list_spin_layer[0][0]))
print((H_mu_nu_list_spin_layer[0][0].bond_dims))

<class 'list'>
<class 'list'>
<class 'pytenet.mpo.MPO'>
[1, 2, 2, 2, 2, 2, 2, 1]


For ground state finding, we use Hatree fock state |11111111110000> as initial state.

In [21]:
#excited states for different molecules could have different total spins, please try different spin sections.
#for water in sto-6g: 1st excited state has spin 1
HFS = generate_single_state(7, [3, 3, 3, 3, 3, 0, 0])
#HFS.orthonormalize('left')
#HFS.orthonormalize('right')


We generate a group of orthogonal Krylov vectors using THC-MPO, with bond dim 40 for Krylov vectors. The vectors are stored in the folder = 'foldername', thus you don't have to generate them again for next time use. 

In [ ]:
N_Krylov_1 = 40
psi_original = copy.deepcopy(HFS)
max_bond_Krylov_1 = 30
#max_bond_Krylov = 75
trunc_tol = 0
foldername_1 = f"../water_Krylov"
#Krylov vectors are included in data, you dont have to run generate it. ofc, you can -regenerate it to verify the algorithm using the following code:
generate_krylov_space_in_disk(N_Krylov_1, H_mu_nu_list_spin_layer, psi_original, max_bond_Krylov_1, trunc_tol, r_thc, foldername_1)

#it indicates that even though during the calculation the bond dims exceed 40, but we only need 37 for Krylov vectors.

Make use of method proposed in https://journals.aps.org/prb/abstract/10.1103/PhysRevB.85.205119 to improve the orthogonality of Krylov vectors. 

In [24]:
H_reduced_non_rotho_1 = generate_reduced_H_non_ortho(N_Krylov_1, foldername_1, mpo_ref)
coeff_1 = coeff_gram_schmidt(N_Krylov_1, foldername_1)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_1 = np.einsum('ik, kl, jl -> ij', coeff_1.conj(), H_reduced_non_rotho_1, coeff_1)

In [25]:
e_ritz_1, v_ritz_1 = solve_ritz(foldername_1, H_reduced_1, N_Krylov_1, coeff_1, max_bond_Krylov_1, e_ground, mpo_ref)

(0.35243292622963907+0j)
(0.02488377003935227+0j)
(0.0021890983286141363+0j)
(0.0002843306280908564+0j)
(0.00023543766282330125+0j)
(0.00022873309136173248+0j)
(0.0001928128917398908+0j)
(0.0001620299822349125+0j)


In [ ]:
N_Krylov_2 = 40
psi_original = copy.deepcopy(v_ritz_1)
max_bond_Krylov_2 = 30
#max_bond_Krylov = 75
trunc_tol = 0
foldername_2 = f"../water_Krylov2"
#Krylov vectors are included in data, you dont have to run generate it. ofc, you can -regenerate it to verify the algorithm using the following code:
generate_krylov_space_in_disk(N_Krylov_2, H_mu_nu_list_spin_layer, psi_original, max_bond_Krylov_2, trunc_tol, r_thc, foldername_2)

#it indicates that even though during the calculation the bond dims exceed 40, but we only need 37 for Krylov vectors.

In [ ]:
H_reduced_non_rotho_2 = generate_reduced_H_non_ortho(N_Krylov_2, foldername_2, mpo_ref)
coeff_2 = coeff_gram_schmidt(N_Krylov_2, foldername_2)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_2 = np.einsum('ik, kl, jl -> ij', coeff_2.conj(), H_reduced_non_rotho_2, coeff_2)

In [ ]:
e_ritz_2, v_ritz_2 = solve_ritz(foldername_2, H_reduced_2, N_Krylov_2, coeff_2, max_bond_Krylov_2, e_ground, mpo_ref)

In [ ]:
# (0.35244435398342944+0j)
# (0.024848970748564625+0j)
# (0.0019585403953357172+0j)
# (7.230799008084432e-05+0j)
# (8.515112028817384e-07+0j)
# (1.4224383448890876e-08+0j)
# (1.0265921446261927e-10+0j)
# (4.263256414560601e-14+0j)